In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('Dinamo_Kharkiv_Vibrani_virshi.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:110]

'Сергій Жадан \nДинамо Харків. Вибрані вірші\n     \n\n     \nСергій Жадан\nДинамо Харків. Вибрані вірші\n     \n     \n'

In [4]:
encoded[:110]

array([ 66,  82,  93,  80, 107,  86,   2,  56,  77,  81,  77,  90,   2,
         0,  54,  85,  90,  77,  89,  91,   2,  70,  77,  93,  87, 107,
        79,   9,   2,  52,  85,  78,  93,  77,  90, 107,   2,  79, 107,
        93, 101, 107,   0,   2,   2,   2,   2,   2,   0,   0,   2,   2,
         2,   2,   2,   0,  66,  82,  93,  80, 107,  86,   2,  56,  77,
        81,  77,  90,   0,  54,  85,  90,  77,  89,  91,   2,  70,  77,
        93,  87, 107,  79,   9,   2,  52,  85,  78,  93,  77,  90, 107,
         2,  79, 107,  93, 101, 107,   0,   2,   2,   2,   2,   2,   0,
         2,   2,   2,   2,   2,   0])

In [5]:
len(vocab)

116

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Создаем генератор, который возвращает пакеты размером
       n_seqs x n_steps из массива arr.
       
       Аргументы
       ---------
       arr: Массив, из которого получаем пакеты
       n_seqs: Batch size, количество последовательностей в пакете
       n_steps: Sequence length, сколько "шагов" делаем в пакете
    '''
    # Считаем количество символов на пакет и количество пакетов, которое можем сформировать
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Сохраняем в массиве только символы, которые позволяют сформировать целое число пакетов
    arr = arr[:n_batches * characters_per_batch]
    
    # Делаем reshape 1D -> 2D, используя n_seqs как число строк, как на картинке
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # пакет данных, который будет подаваться на вход сети
        x = arr[:, n:n+n_steps]
        # целевой пакет, с которым будем сравнивать предсказание, получаем сдвиганием "x" на один символ вперед
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:5, :5])
print('\ny\n', y[:5, :5])

x
 [[ 66  82  93  80 107]
 [ 95  22  93 107  79]
 [  2   2 107   2  79]
 [ 95  85  88  85   2]
 [  0   2   2   2   2]]

y
 [[ 82  93  80 107  86]
 [ 22  93 107  79   8]
 [  2 107   2  79  77]
 [ 85  88  85   2  90]
 [  2   2   2   2   2]]


In [8]:
def build_inputs(batch_size, num_steps):
    ''' Определяем placeholder'ы для входных, целевых данных, а также вероятности drop out
    
        Аргументы
        ---------
        batch_size: Batch size, количество последовательностей в пакете
        num_steps: Sequence length, сколько "шагов" делаем в пакете
        
    '''
    # Объявляем placeholder'ы
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Placeholder для вероятности drop out
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [9]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Строим LSTM ячейку.
    
        Аргументы
        ---------
        keep_prob: Скаляр (tf.placeholder) для dropout keep probability
        lstm_size: Размер скрытых слоев в LSTM ячейках
        num_layers: Количество LSTM слоев
        batch_size: Batch size

    '''
    ### Строим LSTM ячейку
    
    def build_cell(lstm_size, keep_prob):
        # Начинаем с базовой LSTM ячейки
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Добавляем dropout к ячейке
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Стэкируем несколько LSTM слоев для придания глубины нашему deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    # Инициализируем начальное состояние LTSM ячейки
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [10]:
def build_output(lstm_output, in_size, out_size):
    ''' Строим softmax слой и возвращаем результат его работы.
    
        Аргументы
        ---------
        
        x: Входящий от LSTM тензор
        in_size: Размер входящего тензора, (кол-во LSTM юнитов скрытого слоя)
        out_size: Размер softmax слоя (объем словаря)
    
    '''

    # вытягиваем и решэйпим тензор, выполняя преобразование 3D -> 2D
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Соединяем результат LTSM слоев с softmax слоем
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Считаем logit-функцию
    logits = tf.matmul(x, softmax_w) + softmax_b
    # Используем функцию softmax для получения предсказания
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [11]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Считаем функцию потери на основании значений logit-функции и целевых значений.
    
        Аргументы
        ---------
        logits: значение logit-функции
        targets: целевые значения, с которыми сравниваем предсказания
        lstm_size: Количество юнитов в LSTM слое
        num_classes: Количество классов в целевых значениях (размер словаря)
        
    '''
    # Делаем one-hot кодирование целевых значений и решейпим по образу и подобию logits
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Считаем значение функции потери softmax cross entropy loss и возвращаем среднее значение
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [12]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Строим оптимизатор для обучения, используя обрезку градиента.
    
        Arguments:
        loss: значение функции потери
        learning_rate: параметр скорости обучения
    
    '''
    
    # Оптимизатор для обучения, обрезка градиента для контроля "взрывающихся" градиентов
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [13]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # Мы будем использовать эту же сеть для сэмплирования (генерации текста),
        # при этом будем подавать по одному символу за один раз
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Строим LSTM ячейку
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Прогоняем данные через RNN и собираем результаты
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)


In [15]:
batch_size = 50        # Размер пакета
num_steps = 50         # Шагов в пакете
lstm_size = 512         # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [16]:
epochs = 20
# Сохраняться каждый N итераций
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Можно раскомментировать строчку ниже и продолжить обучение с checkpoint'а
   # saver.restore(sess, 'checkpoints/i320_l512.ckpt')
    counter = 0
    for e in range(epochs):
        # Обучаем сеть
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 1/20...  Training Step: 1...  Training loss: 4.7518...  3.4134 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.6222...  3.7337 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.9887...  3.3194 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 7.2948...  3.3844 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 5.6346...  3.5175 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.5640...  3.4214 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.7446...  3.8107 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.5898...  5.5650 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.5120...  4.5823 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.3803...  3.8117 sec/batch
Epoch: 1/2

Epoch: 2/20...  Training Step: 98...  Training loss: 2.8997...  4.3711 sec/batch
Epoch: 2/20...  Training Step: 99...  Training loss: 2.9078...  5.3308 sec/batch
Epoch: 2/20...  Training Step: 100...  Training loss: 2.8783...  4.4992 sec/batch
Epoch: 2/20...  Training Step: 101...  Training loss: 2.9949...  4.2670 sec/batch
Epoch: 2/20...  Training Step: 102...  Training loss: 2.9454...  5.3498 sec/batch
Epoch: 2/20...  Training Step: 103...  Training loss: 2.8641...  6.8018 sec/batch
Epoch: 2/20...  Training Step: 104...  Training loss: 2.8849...  6.0063 sec/batch
Epoch: 2/20...  Training Step: 105...  Training loss: 2.8936...  5.0536 sec/batch
Epoch: 2/20...  Training Step: 106...  Training loss: 2.8305...  4.3681 sec/batch
Epoch: 2/20...  Training Step: 107...  Training loss: 2.9103...  4.5883 sec/batch
Epoch: 2/20...  Training Step: 108...  Training loss: 2.8376...  4.0319 sec/batch
Epoch: 2/20...  Training Step: 109...  Training loss: 2.8700...  3.8547 sec/batch
Epoch: 2/20...  Tr

Epoch: 3/20...  Training Step: 198...  Training loss: 2.5617...  6.7078 sec/batch
Epoch: 4/20...  Training Step: 199...  Training loss: 2.6599...  6.4196 sec/batch
Epoch: 4/20...  Training Step: 200...  Training loss: 2.5408...  5.5169 sec/batch
Epoch: 4/20...  Training Step: 201...  Training loss: 2.4578...  4.1720 sec/batch
Epoch: 4/20...  Training Step: 202...  Training loss: 2.4623...  3.8417 sec/batch
Epoch: 4/20...  Training Step: 203...  Training loss: 2.4909...  3.8627 sec/batch
Epoch: 4/20...  Training Step: 204...  Training loss: 2.3894...  4.4512 sec/batch
Epoch: 4/20...  Training Step: 205...  Training loss: 2.5169...  3.9488 sec/batch
Epoch: 4/20...  Training Step: 206...  Training loss: 2.5082...  4.3461 sec/batch
Epoch: 4/20...  Training Step: 207...  Training loss: 2.5277...  3.8277 sec/batch
Epoch: 4/20...  Training Step: 208...  Training loss: 2.4828...  3.8197 sec/batch
Epoch: 4/20...  Training Step: 209...  Training loss: 2.5065...  3.7717 sec/batch
Epoch: 4/20...  

Epoch: 5/20...  Training Step: 298...  Training loss: 2.3354...  3.9958 sec/batch
Epoch: 5/20...  Training Step: 299...  Training loss: 2.4190...  3.9228 sec/batch
Epoch: 5/20...  Training Step: 300...  Training loss: 2.3980...  3.8687 sec/batch
Epoch: 5/20...  Training Step: 301...  Training loss: 2.2981...  3.8217 sec/batch
Epoch: 5/20...  Training Step: 302...  Training loss: 2.3363...  3.8207 sec/batch
Epoch: 5/20...  Training Step: 303...  Training loss: 2.3618...  4.9955 sec/batch
Epoch: 5/20...  Training Step: 304...  Training loss: 2.2643...  4.3891 sec/batch
Epoch: 5/20...  Training Step: 305...  Training loss: 2.3617...  4.7954 sec/batch
Epoch: 5/20...  Training Step: 306...  Training loss: 2.3104...  4.9725 sec/batch
Epoch: 5/20...  Training Step: 307...  Training loss: 2.3360...  4.4562 sec/batch
Epoch: 5/20...  Training Step: 308...  Training loss: 2.2575...  3.9038 sec/batch
Epoch: 5/20...  Training Step: 309...  Training loss: 2.3181...  4.0829 sec/batch
Epoch: 5/20...  

Epoch: 7/20...  Training Step: 398...  Training loss: 2.2389...  3.8708 sec/batch
Epoch: 7/20...  Training Step: 399...  Training loss: 2.1701...  3.9478 sec/batch
Epoch: 7/20...  Training Step: 400...  Training loss: 2.1790...  3.9188 sec/batch
Epoch: 7/20...  Training Step: 401...  Training loss: 2.2068...  3.6716 sec/batch
Epoch: 7/20...  Training Step: 402...  Training loss: 2.1401...  3.8337 sec/batch
Epoch: 7/20...  Training Step: 403...  Training loss: 2.2747...  3.8257 sec/batch
Epoch: 7/20...  Training Step: 404...  Training loss: 2.2700...  3.8397 sec/batch
Epoch: 7/20...  Training Step: 405...  Training loss: 2.2605...  3.8467 sec/batch
Epoch: 7/20...  Training Step: 406...  Training loss: 2.2138...  3.8277 sec/batch
Epoch: 7/20...  Training Step: 407...  Training loss: 2.2541...  3.8217 sec/batch
Epoch: 7/20...  Training Step: 408...  Training loss: 2.1523...  3.8277 sec/batch
Epoch: 7/20...  Training Step: 409...  Training loss: 2.2179...  3.8487 sec/batch
Epoch: 7/20...  

...

Epoch: 14/20...  Training Step: 895...  Training loss: 1.9788...  4.9905 sec/batch
Epoch: 14/20...  Training Step: 896...  Training loss: 2.0086...  4.5983 sec/batch
Epoch: 14/20...  Training Step: 897...  Training loss: 2.0292...  4.6773 sec/batch
Epoch: 14/20...  Training Step: 898...  Training loss: 1.9051...  4.6983 sec/batch
Epoch: 14/20...  Training Step: 899...  Training loss: 2.0059...  4.9325 sec/batch
Epoch: 14/20...  Training Step: 900...  Training loss: 1.9845...  4.5733 sec/batch
Epoch: 14/20...  Training Step: 901...  Training loss: 2.0028...  4.7023 sec/batch
Epoch: 14/20...  Training Step: 902...  Training loss: 1.9399...  4.5282 sec/batch
Epoch: 14/20...  Training Step: 903...  Training loss: 1.9812...  4.7123 sec/batch
Epoch: 14/20...  Training Step: 904...  Training loss: 2.0203...  5.1847 sec/batch
Epoch: 14/20...  Training Step: 905...  Training loss: 2.0285...  4.6603 sec/batch
Epoch: 14/20...  Training Step: 906...  Training loss: 2.0170...  5.0346 sec/batch
Epoc

Epoch: 16/20...  Training Step: 994...  Training loss: 1.8854...  4.8194 sec/batch
Epoch: 16/20...  Training Step: 995...  Training loss: 1.9179...  4.6804 sec/batch
Epoch: 16/20...  Training Step: 996...  Training loss: 1.8879...  4.7244 sec/batch
Epoch: 16/20...  Training Step: 997...  Training loss: 1.9681...  4.5522 sec/batch
Epoch: 16/20...  Training Step: 998...  Training loss: 2.0347...  5.0176 sec/batch
Epoch: 16/20...  Training Step: 999...  Training loss: 2.0178...  4.4502 sec/batch
Epoch: 16/20...  Training Step: 1000...  Training loss: 1.9626...  4.4882 sec/batch
Epoch: 16/20...  Training Step: 1001...  Training loss: 1.9783...  4.5402 sec/batch
Epoch: 16/20...  Training Step: 1002...  Training loss: 1.8995...  4.8094 sec/batch
Epoch: 16/20...  Training Step: 1003...  Training loss: 1.9494...  4.8114 sec/batch
Epoch: 16/20...  Training Step: 1004...  Training loss: 1.9665...  4.9835 sec/batch
Epoch: 16/20...  Training Step: 1005...  Training loss: 1.9489...  4.5312 sec/batc

Epoch: 17/20...  Training Step: 1092...  Training loss: 1.9409...  4.5622 sec/batch
Epoch: 17/20...  Training Step: 1093...  Training loss: 1.9071...  4.7033 sec/batch
Epoch: 17/20...  Training Step: 1094...  Training loss: 1.9504...  4.6253 sec/batch
Epoch: 17/20...  Training Step: 1095...  Training loss: 1.9360...  4.9085 sec/batch
Epoch: 17/20...  Training Step: 1096...  Training loss: 1.8356...  4.5732 sec/batch
Epoch: 17/20...  Training Step: 1097...  Training loss: 1.9328...  4.6823 sec/batch
Epoch: 17/20...  Training Step: 1098...  Training loss: 1.9235...  4.6723 sec/batch
Epoch: 17/20...  Training Step: 1099...  Training loss: 1.9251...  4.6503 sec/batch
Epoch: 17/20...  Training Step: 1100...  Training loss: 1.8443...  4.8174 sec/batch
Epoch: 17/20...  Training Step: 1101...  Training loss: 1.8967...  4.5362 sec/batch
Epoch: 17/20...  Training Step: 1102...  Training loss: 1.9647...  4.8374 sec/batch
Epoch: 17/20...  Training Step: 1103...  Training loss: 1.9642...  4.7204 se

Epoch: 19/20...  Training Step: 1190...  Training loss: 1.8720...  4.7214 sec/batch
Epoch: 19/20...  Training Step: 1191...  Training loss: 1.8256...  4.6873 sec/batch
Epoch: 19/20...  Training Step: 1192...  Training loss: 1.8053...  4.6693 sec/batch
Epoch: 19/20...  Training Step: 1193...  Training loss: 1.8586...  4.5723 sec/batch
Epoch: 19/20...  Training Step: 1194...  Training loss: 1.8142...  4.6643 sec/batch
Epoch: 19/20...  Training Step: 1195...  Training loss: 1.8875...  4.7684 sec/batch
Epoch: 19/20...  Training Step: 1196...  Training loss: 1.9708...  4.8414 sec/batch
Epoch: 19/20...  Training Step: 1197...  Training loss: 1.9505...  4.5412 sec/batch
Epoch: 19/20...  Training Step: 1198...  Training loss: 1.9116...  4.5492 sec/batch
Epoch: 19/20...  Training Step: 1199...  Training loss: 1.9256...  4.5993 sec/batch
Epoch: 19/20...  Training Step: 1200...  Training loss: 1.8248...  4.7944 sec/batch
Epoch: 19/20...  Training Step: 1201...  Training loss: 1.8923...  4.3831 se

Epoch: 20/20...  Training Step: 1288...  Training loss: 1.8483...  4.8434 sec/batch
Epoch: 20/20...  Training Step: 1289...  Training loss: 1.8919...  4.7043 sec/batch
Epoch: 20/20...  Training Step: 1290...  Training loss: 1.8613...  4.6693 sec/batch
Epoch: 20/20...  Training Step: 1291...  Training loss: 1.8351...  5.0866 sec/batch
Epoch: 20/20...  Training Step: 1292...  Training loss: 1.8934...  4.6943 sec/batch
Epoch: 20/20...  Training Step: 1293...  Training loss: 1.8588...  4.6813 sec/batch
Epoch: 20/20...  Training Step: 1294...  Training loss: 1.7642...  4.7954 sec/batch
Epoch: 20/20...  Training Step: 1295...  Training loss: 1.8627...  4.7354 sec/batch
Epoch: 20/20...  Training Step: 1296...  Training loss: 1.8492...  4.7144 sec/batch
Epoch: 20/20...  Training Step: 1297...  Training loss: 1.8609...  4.7174 sec/batch
Epoch: 20/20...  Training Step: 1298...  Training loss: 1.8119...  4.6893 sec/batch
Epoch: 20/20...  Training Step: 1299...  Training loss: 1.8252...  5.3398 se

In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [18]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="Коли втрачаю я рівновагу, "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [21]:
checkpoint = 'checkpoints/i1320_l512.ckpt'
samp = sample(checkpoint, 1500, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1320_l512.ckpt
Коли втрачаю я рівновагу, яка не падеравовий споста,
     запле наси те замути
     простою
     на поростути триваєш на зному
     і насілом на прості,
     з витрами свіганий мастина.
     Все, що в ньоми з тима приходить світти,
     забивали на всій сверів на вісту
     і ступали,
     і в насі, мов висторито залости.
     І нічерні сталі помарилися, ми в тенбомих
     іністи, що теревона відчу вона води.
     Вироз на все не повани пем’ять – пробовнаю,
     зі своїю толом,
     і стіраюти повертою в кількими не викали толки,
     і все, що потинею в нібо з таких наволонамі.
     
     Помінники мастіманими від нами
     подитуєш сорці не від небі золонне,
     нач скалить витись, як від віч вого на звори,
     і вини повітряла старі,
     наси всі сворина.
     
     
     2001 
     

      
      Поментрів дитяває стирі на знасти в свогому
  о  вікнала стого потравня, як нерідну вітро,
     всім при содцех мор